In [1]:
import pandas as pd
import duckdb as db
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
# Connect to the clinisys_all database
path_to_db = '../../database/clinisys_all.duckdb'

print(f"Connected to database: {path_to_db}")
print(f"Database file exists: {os.path.exists(path_to_db)}")

# Use context manager for automatic connection cleanup
with db.connect(path_to_db, read_only=True) as conn:
    # List all schemas
    schemas = conn.execute("SELECT schema_name FROM information_schema.schemata").fetchdf()
    print("\n=== Available Schemas ===")
    print(schemas)

    # List all tables in silver schema
    silver_tables = conn.execute("""
        SELECT table_name, table_type 
        FROM information_schema.tables 
        WHERE table_schema = 'silver'
        ORDER BY table_name
    """).fetchdf()

    print(f"\n=== Tables in Silver Schema ({len(silver_tables)} tables) ===")
    print(silver_tables)

    # Explore each table in silver schema
    print("\n" + "="*80)
    print("EXPLORING SILVER SCHEMA TABLES")
    print("="*80)

    for idx, row in silver_tables.iterrows():
        table_name = row['table_name']
        table_type = row['table_type']
        
        print(f"\n{'='*60}")
        print(f"TABLE: {table_name} (Type: {table_type})")
        print(f"{'='*60}")
        
        try:
            # Get table schema
            schema_info = conn.execute(f"""
                SELECT column_name, data_type, is_nullable
                FROM information_schema.columns 
                WHERE table_schema = 'silver' AND table_name = '{table_name}'
                ORDER BY ordinal_position
            """).fetchdf()
            
            print(f"\nSchema ({len(schema_info)} columns):")
            # print(schema_info.to_string(index=False))
            
            # Get row count
            count_result = conn.execute(f"SELECT COUNT(*) as row_count FROM silver.{table_name}").fetchdf()
            row_count = count_result['row_count'].iloc[0]
            print(f"\nTotal rows: {row_count:,}")
            
            # Get sample data (first 10 rows)
            if row_count > 0:
                sample_data = conn.execute(f"SELECT * FROM silver.{table_name} LIMIT 10").fetchdf()
                print(f"\nSample data (first 10 rows):")
                print(sample_data.to_string(index=False))
                
                # Show data types and null counts
                print(f"\nData info:")
                print(f"Shape: {sample_data.shape}")
                print(f"Memory usage: {sample_data.memory_usage(deep=True).sum() / 1024:.2f} KB")
                
                # Show null counts for each column
                null_counts = sample_data.isnull().sum()
                if null_counts.sum() > 0:
                    print(f"\nNull counts in sample:")
                    for col, null_count in null_counts.items():
                        if null_count > 0:
                            print(f"  {col}: {null_count}")
            else:
                print("\nTable is empty - no sample data to show")
                
        except Exception as e:
            print(f"Error exploring table {table_name}: {str(e)}")
        
        print(f"\n{'-'*60}")

print(f"\n{'='*80}")
print("EXPLORATION COMPLETE")
print(f"{'='*80}")

Connected to database: ../../database/clinisys_all.duckdb
Database file exists: True

=== Available Schemas ===
          schema_name
0              bronze
1                gold
2                main
3              silver
4  information_schema
5                main
6          pg_catalog
7                main

=== Tables in Silver Schema (20 tables) ===
                           table_name  table_type
0         view_congelamentos_embrioes  BASE TABLE
1           view_congelamentos_ovulos  BASE TABLE
2            view_congelamentos_semen  BASE TABLE
3     view_congelamentos_semen_doador  BASE TABLE
4      view_descongelamentos_embrioes  BASE TABLE
5        view_descongelamentos_ovulos  BASE TABLE
6            view_embrioes_congelados  BASE TABLE
7                         view_exames  BASE TABLE
8   view_extrato_atendimentos_central  BASE TABLE
9                   view_medicamentos  BASE TABLE
10      view_medicamentos_prescricoes  BASE TABLE
11                       view_medicos  BASE T

In [3]:
#!/usr/bin/env python3
"""
Extract Prontuario Data - Modular Functions
Extract data for a specific prontuario from finops tables with individual functions for each table.
"""

import pandas as pd
import duckdb as db
import os

def get_database_connection():
    """
    Create and return a connection to the clinisys_all database
    
    Returns:
        duckdb.DuckDBPyConnection: Database connection
    """
    path_to_db = '../../database/clinisys_all.duckdb'
    conn = db.connect(path_to_db, read_only=True)
    
    print(f"Connected to database: {path_to_db}")
    print(f"Database file exists: {os.path.exists(path_to_db)}")
    
    return conn

def get_database_context():
    """
    Create a context manager for the clinisys_all database connection
    
    Returns:
        context manager: Database connection context manager
    """
    path_to_db = '../../database/clinisys_all.duckdb'
    return db.connect(path_to_db, read_only=True)

def extract_orcamentos_data(conn, target_prontuario):
    """
    Extract orcamentos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing orcamentos data
    """
    print(f"\n2. EXTRACTING VIEW_ORCAMENTOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_orc = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_orcamentos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_orcamentos columns: {list(schema_orc['column_name'])}")
        
        # Extract data
        orcamentos_df = conn.execute(f"""
            SELECT * FROM silver.view_orcamentos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(orcamentos_df)} records in view_orcamentos")
        if len(orcamentos_df) > 0:
            print("Sample data:")
            print(orcamentos_df.head())
            print(f"Columns: {list(orcamentos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting orcamentos data: {str(e)}")
        orcamentos_df = pd.DataFrame()
    
    return orcamentos_df

def extract_extrato_atendimento_central_data(conn, target_prontuario):
    """
    Extract extrato_atendimento_central data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing extrato_atendimento_central data
    """
    print(f"\n3. EXTRACTING VIEW_EXTRATO_ATENDIMENTO_CENTRAL DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_eac = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_extrato_atendimentos_central'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_extrato_atendimento_central columns: {list(schema_eac['column_name'])}")
        
        # Extract data
        extrato_atendimento_central_df = conn.execute(f"""
            SELECT * FROM silver.view_extrato_atendimentos_central 
            WHERE prontuario = {target_prontuario}
            ORDER BY data DESC, inicio DESC
        """).fetchdf()
        
        print(f"Found {len(extrato_atendimento_central_df)} records in view_extrato_atendimento_central")
        if len(extrato_atendimento_central_df) > 0:
            print("Sample data:")
            print(extrato_atendimento_central_df.head())
            print(f"Columns: {list(extrato_atendimento_central_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting extrato_atendimento_central data: {str(e)}")
        extrato_atendimento_central_df = pd.DataFrame()
    
    return extrato_atendimento_central_df

def extract_tratamentos_data(conn, target_prontuario):
    """
    Extract tratamentos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing tratamentos data
    """
    print(f"\n4. EXTRACTING VIEW_TRATAMENTOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_trat = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_tratamentos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_tratamentos columns: {list(schema_trat['column_name'])}")
        
        # Extract data
        tratamentos_df = conn.execute(f"""
            SELECT * FROM silver.view_tratamentos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(tratamentos_df)} records in view_tratamentos")
        if len(tratamentos_df) > 0:
            print("Sample data:")
            print(tratamentos_df.head())
            print(f"Columns: {list(tratamentos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting tratamentos data: {str(e)}")
        tratamentos_df = pd.DataFrame()
    
    return tratamentos_df

def extract_congelamentos_embrioes_data(conn, target_prontuario):
    """
    Extract congelamentos_embrioes data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing congelamentos_embrioes data
    """
    print(f"\n5. EXTRACTING VIEW_CONGELAMENTOS_EMBRIOES DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_ce = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_congelamentos_embrioes'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_congelamentos_embrioes columns: {list(schema_ce['column_name'])}")
        
        # Extract data
        congelamentos_embrioes_df = conn.execute(f"""
            SELECT * FROM silver.view_congelamentos_embrioes 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(congelamentos_embrioes_df)} records in view_congelamentos_embrioes")
        if len(congelamentos_embrioes_df) > 0:
            print("Sample data:")
            print(congelamentos_embrioes_df.head())
            print(f"Columns: {list(congelamentos_embrioes_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting congelamentos_embrioes data: {str(e)}")
        congelamentos_embrioes_df = pd.DataFrame()
    
    return congelamentos_embrioes_df

def extract_congelamentos_ovulos_data(conn, target_prontuario):
    """
    Extract congelamentos_ovulos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing congelamentos_ovulos data
    """
    print(f"\n6. EXTRACTING VIEW_CONGELAMENTOS_OVULOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_co = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_congelamentos_ovulos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_congelamentos_ovulos columns: {list(schema_co['column_name'])}")
        
        # Extract data
        congelamentos_ovulos_df = conn.execute(f"""
            SELECT * FROM silver.view_congelamentos_ovulos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(congelamentos_ovulos_df)} records in view_congelamentos_ovulos")
        if len(congelamentos_ovulos_df) > 0:
            print("Sample data:")
            print(congelamentos_ovulos_df.head())
            print(f"Columns: {list(congelamentos_ovulos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting congelamentos_ovulos data: {str(e)}")
        congelamentos_ovulos_df = pd.DataFrame()
    
    return congelamentos_ovulos_df

def extract_descongelamentos_embrioes_data(conn, target_prontuario):
    """
    Extract descongelamentos_embrioes data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing descongelamentos_embrioes data
    """
    print(f"\n7. EXTRACTING VIEW_DESCONGELAMENTOS_EMBRIOES DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_de = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_descongelamentos_embrioes'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_descongelamentos_embrioes columns: {list(schema_de['column_name'])}")
        
        # Extract data
        descongelamentos_embrioes_df = conn.execute(f"""
            SELECT * FROM silver.view_descongelamentos_embrioes 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(descongelamentos_embrioes_df)} records in view_descongelamentos_embrioes")
        if len(descongelamentos_embrioes_df) > 0:
            print("Sample data:")
            print(descongelamentos_embrioes_df.head())
            print(f"Columns: {list(descongelamentos_embrioes_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting descongelamentos_embrioes data: {str(e)}")
        descongelamentos_embrioes_df = pd.DataFrame()
    
    return descongelamentos_embrioes_df

def extract_descongelamentos_ovulos_data(conn, target_prontuario):
    """
    Extract descongelamentos_ovulos data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing descongelamentos_ovulos data
    """
    print(f"\n8. EXTRACTING VIEW_DESCONGELAMENTOS_OVULOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_do = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_descongelamentos_ovulos'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_descongelamentos_ovulos columns: {list(schema_do['column_name'])}")
        
        # Extract data
        descongelamentos_ovulos_df = conn.execute(f"""
            SELECT * FROM silver.view_descongelamentos_ovulos 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(descongelamentos_ovulos_df)} records in view_descongelamentos_ovulos")
        if len(descongelamentos_ovulos_df) > 0:
            print("Sample data:")
            print(descongelamentos_ovulos_df.head())
            print(f"Columns: {list(descongelamentos_ovulos_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting descongelamentos_ovulos data: {str(e)}")
        descongelamentos_ovulos_df = pd.DataFrame()
    
    return descongelamentos_ovulos_df

def extract_embrioes_congelados_data(conn, target_prontuario):
    """
    Extract embrioes_congelados data for a specific prontuario
    
    Args:
        conn: Database connection
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        pd.DataFrame: DataFrame containing embrioes_congelados data
    """
    print(f"\n9. EXTRACTING VIEW_EMBRIOES_CONGELADOS DATA")
    print("-" * 50)
    
    try:
        # Check schema first
        schema_ec = conn.execute("""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_schema = 'silver' AND table_name = 'view_embrioes_congelados'
            ORDER BY ordinal_position
        """).fetchdf()
        print(f"view_embrioes_congelados columns: {list(schema_ec['column_name'])}")
        
        # Extract data
        embrioes_congelados_df = conn.execute(f"""
            SELECT * FROM silver.view_embrioes_congelados 
            WHERE prontuario = {target_prontuario}
            ORDER BY id DESC
        """).fetchdf()
        
        print(f"Found {len(embrioes_congelados_df)} records in view_embrioes_congelados")
        if len(embrioes_congelados_df) > 0:
            print("Sample data:")
            print(embrioes_congelados_df.head())
            print(f"Columns: {list(embrioes_congelados_df.columns)}")
        else:
            print("No data found for this prontuario")
            
    except Exception as e:
        print(f"Error extracting embrioes_congelados data: {str(e)}")
        embrioes_congelados_df = pd.DataFrame()
    
    return embrioes_congelados_df

def extract_prontuario_data(target_prontuario=876950):
    """
    Extract all data for a specific prontuario from the finops tables:
    - view_orcamentos  
    - view_extrato_atendimento_central
    - view_tratamentos
    - view_congelamentos_embrioes
    - view_congelamentos_ovulos
    - view_descongelamentos_embrioes
    - view_descongelamentos_ovulos
    - view_embrioes_congelados
    
    Args:
        target_prontuario (int): The prontuario number to extract data for
        
    Returns:
        tuple: Eight DataFrames containing the extracted data
    """
    
    print(f"\n{'='*80}")
    print(f"EXTRACTING DATA FOR PRONTUARIO: {target_prontuario}")
    print(f"{'='*80}")
    
    # Use context manager for automatic connection cleanup
    with get_database_context() as conn:
        # Extract data from each table using individual functions
        orcamentos_df = extract_orcamentos_data(conn, target_prontuario)
        extrato_atendimento_central_df = extract_extrato_atendimento_central_data(conn, target_prontuario)
        tratamentos_df = extract_tratamentos_data(conn, target_prontuario)
        congelamentos_embrioes_df = extract_congelamentos_embrioes_data(conn, target_prontuario)
        congelamentos_ovulos_df = extract_congelamentos_ovulos_data(conn, target_prontuario)
        descongelamentos_embrioes_df = extract_descongelamentos_embrioes_data(conn, target_prontuario)
        descongelamentos_ovulos_df = extract_descongelamentos_ovulos_data(conn, target_prontuario)
        embrioes_congelados_df = extract_embrioes_congelados_data(conn, target_prontuario)
        
        # Summary
        print(f"\n{'='*80}")
        print(f"SUMMARY FOR PRONTUARIO {target_prontuario}")
        print(f"{'='*80}")
        print(f"view_orcamentos: {len(orcamentos_df)} records")
        print(f"view_extrato_atendimento_central: {len(extrato_atendimento_central_df)} records")
        print(f"view_tratamentos: {len(tratamentos_df)} records")
        print(f"view_congelamentos_embrioes: {len(congelamentos_embrioes_df)} records")
        print(f"view_congelamentos_ovulos: {len(congelamentos_ovulos_df)} records")
        print(f"view_descongelamentos_embrioes: {len(descongelamentos_embrioes_df)} records")
        print(f"view_descongelamentos_ovulos: {len(descongelamentos_ovulos_df)} records")
        print(f"view_embrioes_congelados: {len(embrioes_congelados_df)} records")
        
        print(f"\n{'='*80}")
        print("DATA EXTRACTION COMPLETE")
        print(f"{'='*80}")
        print("DataFrames available:")
        print("- orcamentos_df") 
        print("- extrato_atendimento_central_df")
        print("- tratamentos_df")
        print("- congelamentos_embrioes_df")
        print("- congelamentos_ovulos_df")
        print("- descongelamentos_embrioes_df")
        print("- descongelamentos_ovulos_df")
        print("- embrioes_congelados_df")
        
        return (orcamentos_df, extrato_atendimento_central_df, tratamentos_df, 
                congelamentos_embrioes_df, congelamentos_ovulos_df, 
                descongelamentos_embrioes_df, descongelamentos_ovulos_df, 
                embrioes_congelados_df)

def find_in_diario_by_all_ids(prontuario: int) -> pd.DataFrame:
    """
    Find all records in diario_vendas by all ids in the patient_id list.
    
    Args:
        patient_id (int): The patient's id
        
    Returns:
        pd.DataFrame: DataFrame containing all diario_vendas records for the patient
    """
    HUNTINGTON_DB_PATH = '../../database/huntington_data_lake.duckdb'
    
    try:
        # Then, get all records from diario_vendas for these codigos
        with db.connect(HUNTINGTON_DB_PATH, read_only=True) as con:
            # Create the IN clause for the query
           
            query_diario = f"""
            SELECT *
            FROM silver.mesclada_vendas
            WHERE "prontuario" = {prontuario}
            ORDER BY "DT Emissao" DESC
            """
            
            df_diario = con.execute(query_diario).df()

            # df_diario['flag_same_prontuario'] = df_diario['Cliente']==prontuario
            
            if df_diario.empty:
                print(f"No records found in silver.mesclada_vendas for prontuario {prontuario}")
                return df_diario
            
            print(f"Found {len(df_diario)} records in silver.mesclada_vendas for prontuario {prontuario}")
            return df_diario
            

    except Exception as e:
        print(f"Error retrieving diario_vendas data for prontuario {prontuario}: {e}")
        return pd.DataFrame()

# if __name__ == "__main__":
#     # Example usage
#     (orcamentos_df, extrato_atendimento_central_df, tratamentos_df, 
#      congelamentos_embrioes_df, congelamentos_ovulos_df, 
#      descongelamentos_embrioes_df, descongelamentos_ovulos_df, 
#      embrioes_congelados_df) = extract_prontuario_data(876950)
    
#     # You can now work with the DataFrames
#     print("\n DataFrames loaded successfully!")
#     print(f"orcamentos_df shape: {orcamentos_df.shape}")
#     print(f"extrato_atendimento_central_df shape: {extrato_atendimento_central_df.shape}")
#     print(f"tratamentos_df shape: {tratamentos_df.shape}")
#     print(f"congelamentos_embrioes_df shape: {congelamentos_embrioes_df.shape}")
#     print(f"congelamentos_ovulos_df shape: {congelamentos_ovulos_df.shape}")
#     print(f"descongelamentos_embrioes_df shape: {descongelamentos_embrioes_df.shape}")
#     print(f"descongelamentos_ovulos_df shape: {descongelamentos_ovulos_df.shape}")
#     print(f"embrioes_congelados_df shape: {embrioes_congelados_df.shape}")
    
#     # Example of using the diario function
#     diario_df = find_in_diario_by_all_ids(876950)
#     print(f"diario_df shape: {diario_df.shape}") 

In [4]:

patient_id = 749579  

(orcamentos_df, extrato_atendimento_central_df, tratamentos_df,
congelamentos_embrioes_df, congelamentos_ovulos_df,
descongelamentos_embrioes_df, descongelamentos_ovulos_df,
embrioes_congelados_df) = extract_prontuario_data(patient_id)

diario_df = find_in_diario_by_all_ids(patient_id)


EXTRACTING DATA FOR PRONTUARIO: 749579

2. EXTRACTING VIEW_ORCAMENTOS DATA
--------------------------------------------------
view_orcamentos columns: ['id', 'prontuario', 'paciente', 'clinica', 'tipo_cotacao', 'profissional', 'status', 'status_entrega', 'nome_contato', 'telefone_contato', 'email_contato', 'comentario_para_paciente', 'comentario_do_paciente', 'orcamento_texto', 'descricao', 'fornecedor', 'qtd_cotada', 'unidade', 'valor_unidade', 'total', 'data_entrega', 'centro_custos', 'valor_total', 'forma', 'parcelas', 'comentarios', 'forma_parcela', 'valor', 'data_pagamento', 'descricao_pagamento', 'data', 'responsavel', 'data_entrega_orcamento', 'data_ultima_modificacao', 'hash', 'extraction_timestamp']
Found 5 records in view_orcamentos
Sample data:
      id  prontuario paciente     clinica tipo_cotacao profissional  \
0  38708      749579   esposa  0000000005         None           21   
1  36463      749579   esposa  0000000005         None           21   
2  33293      749579

In [5]:
orcamentos_df

,id,prontuario,paciente,clinica,tipo_cotacao,profissional,status,status_entrega,nome_contato,telefone_contato,email_contato,comentario_para_paciente,comentario_do_paciente,orcamento_texto,descricao,fornecedor,qtd_cotada,unidade,valor_unidade,total,data_entrega,centro_custos,valor_total,forma,parcelas,comentarios,forma_parcela,valor,data_pagamento,descricao_pagamento,data,responsavel,data_entrega_orcamento,data_ultima_modificacao,hash,extraction_timestamp
0,38708,749579,esposa,0000000005,None,21,Telefone,Realizado,None,None,None,<p><strong>10/07/24 </strong>Hoje a paciente e...,None,<p><img alt=&quot;&quot; src=&quot;data:image/...,0000075997|@|,|@|,|@|,<NA>,|@|,|@|,NaT,1,None,Dinheiro,None,None,None,NaN,NaT,None,2024-07-10,5814,2024-07-10,2024-12-19,ae43f521db654534fab823f585998d87,2025-07-21 21:40:47
1,36463,749579,esposa,0000000005,None,21,Pessoalmente,Realizado,None,None,None,<p><strong>02/05/2<span style=&quot;font-size:...,None,<p><img alt=&quot;&quot; src=&quot;data:image/...,0000760442|@|,|@|,|@|,<NA>,"12.120,70|@|",|@|,NaT,1,None,Dinheiro,None,None,None,NaN,NaT,None,2024-05-02,6162,2024-05-02,2024-06-26,976cab79e99e0a8d91afd6f6e4600015,2025-07-21 21:40:47
2,33293,749579,esposa,0000000005,None,645,Pessoalmente,Realizado,None,None,None,<p><strong>29/01/2024 -</strong> Apresento est...,<div style=&quot;page-break-after: always&quot...,<p><img alt=&quot;&quot; src=&quot;data:image/...,0000075995|@|,|@|,|@|,<NA>,"9.787,72|@|",|@|,NaT,1,None,Dinheiro,None,None,None,NaN,NaT,None,2024-01-29,6162,2024-01-29,2024-04-19,492d54e9a659ae1f567c740cacdc846b,2025-07-21 21:40:47
3,28307,749579,esposa,0000000005,None,645,Pessoalmente,Realizado,None,None,None,<p><strong>08/08/23</strong> Apresento estimat...,<p>&nbsp;</p>\r\n\r\n<p>&nbsp;</p>\r\n,<p><img alt=&quot;&quot; src=&quot;data:image/...,0000075997|@|,|@|,|@|,<NA>,|@|,|@|,NaT,1,None,Dinheiro,None,None,None,NaN,NaT,None,2023-09-13,5814,NaT,2024-01-16,a3c1e7f3f4f68bc226d2650355dc6f16,2025-07-21 21:40:47
4,22967,749579,esposa,0000000005,None,645,Pessoalmente,Não Realizado,None,None,None,None,None,<p>26/01/23 -&nbsp;Paciente Daniela realizou U...,0000076119|@|,|@|,|@|,<NA>,|@|,|@|,NaT,1,None,Dinheiro,None,None,None,NaN,NaT,None,2023-01-26,5290,NaT,2023-12-19,d2bc999113babf97937ce4275fc16ca9,2025-07-21 21:40:47


In [6]:
tratamentos_df 

,id,prontuario,unidade,idade_esposa,idade_marido,paciente_tratamento,tentativa,data_procedimento,hora_procedimento,tipo_procedimento,motivo_congelamento,tipo_inducao,protocolo,tipo_ciclo_congelado,tipo_preparo_endometrial,num_congelados,num_transferir,origem_ovulo,status_ovulo,prontuario_doadora,origem_espermatozoide,fonte_espermatozoide,status_espermatozoide,origem_material,material_heterologo,utero_substituicao,prontuario_genitores,prontuario_uter_substituicao,doacao_ovulos,destino_doacao_ovulos,prontuario_receptora,complemento_receptora,doacao_embrioes,era,emma,alice,matrice_lab,imunohistoquimica,outros_biopsia,consentimento,financeiro,embryoscope,pgt,era_auxiliar,recomendacao_transferencia,resultado_era,emma_resultado,alice_resultado,picsi,hatching_od,hatching_oe,medicacao_parental,peso_paciente,altura_paciente,fumante_paciente,data_dum,peso_conjuge,altura_conjuge,fumante_conjuge,fator_infertilidade1,fator_infertilidade2,fator_infertilidade3,fsh_exame,foliculos_antrais,data_histeroscopia,data_inicio_inducao,dias_inducao_preparo,paciente_inducao,ropa,data_inicio_agonista,gnrh,fsh_agonista,lh_agonista,outros_medicamentos,gonal,menogon,menopur,pergoveris,elonva,serofene_clomid_indux,luveris,puregon,letrozole,fostimon,merional,rekovelle,descricao_outras_medicacoes,dose_total_fsh,dose_total_lh,dose_total_fsh_lh,coorte_e2,coorte_aco,coorte_progesterona,coq10,hormonal_combinado,coorte_testosterona,agonista_coorte,coorte_antagonista,coorte_dhea,utilizado_hcg,motivo_cancelamento_hcg,data_hcg,horario_planejado_hcg,horario_real_hcg,ovidrel,gonapeptyl,dose_gonapeptyl,outras_complicacoes_hcg,dose_outras_complicacoes,lupron,outros_hcg,descricao_outros_hcg,data_inicio_progesterona,p4_inicio_progesterona,estradiol_fase_lutea,progesterona_fase_lutea,progesterona_oral,progesterona_vaginal,progesterona_injetavel,combinacao_oral_vaginal,nenhuma_progesterona,hcg_progesterona,data_congelamento,metodo_descongelamento,valor_palheta,manter_excedentes,observacoes_descongelamento,dia_transferencia,data_transferencia,medida_endometrio,padrao_endometrio,p4_transferencia,motivo_nao_transferir,motivo_cancelamento_tratamento,ligacao_enfermagem,beta_valor,beta_valor2,data_ultrassom1,data_ultrassom2,num_sacos_gestacionais,tsh,data_ultrassom1_contato,descricao_ultrassom1,bcf_embrião1_status,bcf_embrião1_fonte,data_ultrassom2_contato,descricao_ultrassom2,data_ultrassom3_contato,descricao_ultrassom3,resultado_tratamento,analise_genetica,resultado_analise_genetica,motivo_perda_aborto,data_nascimento_bebes,obstetra,nome_bebe1,sexo_bebe1,peso_bebe1,altura_bebe1,idade_gestacional_bebe1,via_parto_bebe1,viabilidade_bebe1,anomalias_bebe1,nome_bebe2,sexo_bebe2,peso_bebe2,altura_bebe2,idade_gestacional_bebe2,via_parto_bebe2,viabilidade_bebe2,anomalias_bebe2,nome_bebe3,sexo_bebe3,peso_bebe3,altura_bebe3,idade_gestacional_bebe3,via_parto_bebe3,viabilidade_bebe3,anomalias_bebe3,nome_bebe4,sexo_bebe4,peso_bebe4,altura_bebe4,idade_gestacional_bebe4,via_parto_bebe4,viabilidade_bebe4,anomalias_bebe4,observacoes_bebes,observacoes_gerais,usuario_responsavel,responsavel_informacoes,hash,extraction_timestamp
0,30408,749579,5,37,30,esposa,8,NaT,None,Ciclo de Congelados,None,None,None,None,Hormonal,None,None,None,None,NaN,None,None,None,Homólogo,None,Não,NaN,NaN,None,None,NaN,None,None,None,None,None,None,None,None,None,None,None,PGT-A,Não,None,None,Não,Não,None,None,None,Não,"65,00","1,62",Não,2024-10-21,"75,00","1,72",None,Fator masculino,None,None,None,None,None,2024-10-21,None,esposa,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não utilizado,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Quantidade de embriões,02,None,ET DOS 2 MELHORES,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Não,None,None,None,None,None,None,None,None,None,None,

In [7]:
extrato_atendimento_central_df[extrato_atendimento_central_df['confirmado'] == 1]

,agendamento_id,data,inicio,data_agendamento_original,medico,medico2,prontuario,evento,evento2,centro_custos,agenda,chegou,confirmado,paciente_codigo,paciente_nome,medico_nome,medico_sobrenome,medico2_nome,centro_custos_nome,agenda_nome,procedimento_nome,hash,extraction_timestamp
0,1306952,2024-12-19,1900-01-01 11:30:00,NaT,5299,<NA>,749579.0,765444,None,5,370,Atendido,1,749579,Daniela Alves Amorim Pelucio,Gabriella,de Oliveira Ferreira,None,6. HTT Brasília,Sala de Ultrassom 01 - BRASILIA,1º US Obstétrico: Confirmação,910a9ad9c35a5a0098f46f05ac02b8a3,2025-07-21 21:39:24
2,1282463,2024-11-11,1900-01-01 11:30:00,NaT,3664,<NA>,749579.0,765568,None,5,139,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Livia Oliveira Munhoz Soares - BRASILIA,FET: Ciclo Próprio,c31c24f8bc5698f178aaef5185e3ecc7,2025-07-21 21:39:24
3,1282462,2024-11-11,1900-01-01 11:30:00,NaT,<NA>,21,749579.0,765568,None,5,368,Atendido,1,749579,Daniela Alves Amorim Pelucio,None,None,Livia Munhoz,6. HTT Brasília,Sala de Procedimento 01 - BRASILIA,FET: Ciclo Próprio,05cfa7e361eb69fdd7aad08d21241002,2025-07-21 21:39:24
6,1279131,2024-11-04,1900-01-01 14:00:00,NaT,3664,<NA>,749579.0,765446,None,5,370,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Sala de Ultrassom 01 - BRASILIA,2º US Ciclo,cbf7b506fabf3649cef0093aa9e82499,2025-07-21 21:39:24
7,1279132,2024-11-04,1900-01-01 14:00:00,NaT,3664,<NA>,749579.0,765446,None,5,139,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Livia Oliveira Munhoz Soares - BRASILIA,2º US Ciclo,02c6e1afce6c47136fce4793d7096eee,2025-07-21 21:39:24
8,1274250,2024-11-01,1900-01-01 10:00:00,NaT,3664,<NA>,749579.0,765562,None,5,139,Desmarcou,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Livia Oliveira Munhoz Soares - BRASILIA,Avaliação clínica (sem cobrança),34ad840e6dfd3edd1f7cb3ba6dafcc35,2025-07-21 21:39:24
11,1277545,2024-10-30,1900-01-01 13:00:00,NaT,3664,<NA>,749579.0,765435,None,5,434,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Sala de Ultrassom 02 - BRASILIA,1° US Ciclo,906fc8d6f8445c3b99d8de5b00dc7673,2025-07-21 21:39:24
12,1277546,2024-10-30,1900-01-01 13:00:00,NaT,3664,<NA>,749579.0,765435,None,5,139,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Livia Oliveira Munhoz Soares - BRASILIA,1° US Ciclo,480db60e0f414157dcd6504ed5e6ecbf,2025-07-21 21:39:24
13,1259916,2024-09-27,1900-01-01 11:00:00,NaT,7627,<NA>,749579.0,79092,None,5,462,Atendido,1,749579,Daniela Alves Amorim Pelucio,Renata,Gonçalves Teixeira Barbosa,None,6. HTT Brasília,Renata Gonçalves - BRASILIA,Consulta de Acompanhamento Psicológico ***,5efe88db21cd1c4eae4b16113dc44d86,2025-07-21 21:39:24
14,1247685,2024-09-27,1900-01-01 10:00:00,NaT,3664,<NA>,749579.0,765562,None,5,139,Atendido,1,749579,Daniela Alves Amorim Pelucio,Livia,Munhoz,None,6. HTT Brasília,Livia Oliveira Munhoz Soares - BRASILIA,Avaliação clínica (sem cobrança),7004e3df15e121d0a04bec1ed389d416,2025-07-21 21:39:24


In [8]:
congelamentos_embrioes_df

,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,CicloRecongelamento,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NEmbrioes,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Ovulo,D2,D3,D4,D5,D6,D7,rack,rack2,rack3,rack4,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,responsavel_congelamento_d5,responsavel_checagem_d5,responsavel_congelamento_d6,responsavel_checagem_d6,responsavel_congelamento_d7,responsavel_checagem_d7,hash,extraction_timestamp
0,26495,E2257/24,5,749579,esposa,2024-09-03,1900-01-01 15:50:00,2877/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,24,24,1534.SD.506,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,NaN,NaN,None,NaN,NaN,NaN,BIOPSIA/TUBBING/CONGELAMENTO D6 REALIZADO POR ...,<NA>,Temizio Rodrigues Pereira,Karen Ferreira Vale Lopes,None,5293,7358,6131,3772,<NA>,<NA>,e888191887e76e2181342e1f62b2565c,2025-07-21 21:38:47
1,25435,E1205/24,5,749579,esposa,2024-05-15,1900-01-01 11:30:00,1479/24,Não,None,None,None,<NA>,NaT,<NA>,NaT,6,6,FOT.1242.SD.PGTA.ES,None,None,None,Vitrificação,NaN,NaN,NaN,NaN,1.0,NaN,NaN,None,NaN,NaN,NaN,PACIENTE SOLICITOU ANÁLISE DE 5 AMOSTRAS\r\nPa...,<NA>,Temizio Rodrigues Pereira,Zeisiane Landim,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,e2e74129f4ede08cb9d29eb9f21eb28d,2025-07-21 21:38:47
2,23815,E21477/23,5,749579,esposa,2023-11-17,1900-01-01 17:00:00,None,Não,None,None,None,<NA>,NaT,<NA>,NaT,1,1,1242.REBIO,B2E,2,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,NaN,NaN,NaN,REBIOPSIA,<NA>,Beatriz de Mattos Silva,None,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,c818ac773833c68679d96edc8cb2ba43,2025-07-21 21:38:47
3,23620,E21285/23,5,749579,esposa,2023-10-30,1900-01-01 12:00:00,2745/23,Não,None,None,None,<NA>,NaT,<NA>,NaT,5,5,1242,B2E,2,None,Vitrificação,NaN,NaN,NaN,NaN,NaN,1.0,NaN,None,NaN,NaN,NaN,RAQUE METÁLICA,<NA>,Manuela Almeida,Temizio Rodrigues Pereira,None,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,dca7f990e56990b64763505df5ec55d9,2025-07-21 21:38:47


In [9]:
descongelamentos_embrioes_df

,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Transferencia,DataTransferencia,Prateleira,Incubadora,transferidos_transferencia,cateter_transferencia,lote_transferencia,validade_transferencia,intercorrencia_transferencia,sangue_interno_transferencia,sangue_externo_transferencia,retorno_transferencia,vezes_retorno_transferencia,Transfer_D5,responsavel_transferencia,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,16283,2425/24,5,749579,0,2024-09-03,2024-11-11,2877/24,1534.SD.506,E2257/24,B2E,5,2,DESVITRIFICAÇÃO,<NA>,2024-11-11,<NA>,K-System,2,WALLACE,619033207,18/10/2026,None,Não,Não,Não,None,None,5293.0,None,None,None,b3712b31ae395a0bd1abac5562c9f229,2025-07-21 21:38:51
1,15138,1332/24,5,749579,0,2024-05-15,2024-06-26,1479/24,FOT.1242.SD.PGTA.ES,E1205/24,B2E,4,2,DESVITRIFICAÇÃO,<NA>,2024-06-26,<NA>,None,2,wallace,None,None,None,Não,Não,Não,None,None,5293.0,None,None,None,b4af18b636c1c198071e59155e988ef0,2025-07-21 21:38:51
2,14581,806/24,5,749579,0,2023-11-17,2024-04-19,2745/23,1242.REBIO,E21477/23,B2E,2,1,desvitrificção,<NA>,2024-04-19,<NA>,None,1,wallace,619023662,12/03/2026,None,Não,Não,Não,0,None,6131.0,None,None,None,b87957c667403ed07e28e46144013534,2025-07-21 21:38:51
3,13755,37/24,5,749579,0,2023-10-30,2024-01-16,2745/23,1242,E21285/23,B2E,2,1,DESVITRIFICAÇÃO,<NA>,2024-01-16,<NA>,None,1,SIDNEY,15068396,14/11/2025,None,Não,Não,Não,None,None,5293.0,None,None,None,192077d1a610e419a9fd70fdf24b9972,2025-07-21 21:38:51
4,13318,1917/23,5,749579,0,2023-10-30,2023-11-16,2745/23,1242,E21285/23,B2E,2,1,DESVITRIFICAÇÃO,<NA>,2023-11-16,<NA>,Thermo 3110 1,None,None,None,None,None,None,None,None,None,None,NaN,REBIOPSIADO POR BEATRIZ,None,None,73134a2eb3852aef4eb29af070b1f9ad,2025-07-21 21:38:51


In [10]:
embrioes_congelados_df

,id,id_oocito,id_congelamento,id_descongelamento,prontuario,pailletes,pailletes_id,cores,embriao,doado,lote,celulas,qualidade,qualidade_recongelamento,comentarios,comentarios_descongelamento,ah,dia2,maturidade,dia2_2,dia2_3,dia3,dia3_2,dia3_3,dia4,dia4_2,dia4_3,dia4_4,gd,gd2,gd3,dia5,dia5_2,dia5_3,dia5_4,dia6,dia6_2,dia6_3,dia6_4,dia7,dia7_2,dia7_3,dia7_4,clivou_d2,clivou_d3,clivou_d4,clivou_d5,clivou_d6,clivou_d7,transferidos,pgd,resultado_pgd,colapso,pgd_congelamento,resultado_pgd_congelamento,mito_teste_pgd_congelamento,dia_congelamento,score_maia,tanque_amostra,caneca_amostra,rack_amostra,observacao,destino,hash,extraction_timestamp
0,76558,214292,26495,0,749579,39,amarela,<NA>,39,None,None,None,Blastocisto Grau 5 - A - A,<NA>,RAQUI METÁLICA (5),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,5.6,D6,1.88,B2E,1,27,None,<NA>,6fc12b28b2377c603b441657e4d2d1bc,2025-07-21 21:38:57
1,76557,214289,26495,0,749579,36,amarela,<NA>,36,None,None,None,Blastocisto Grau 5 - B - B,<NA>,RAQUI METÁLICA (5),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,5.6,D6,"0,7",B2E,1,27,None,<NA>,4700368f2e2ec71191227aa9ef426e2b,2025-07-21 21:38:57
2,76556,214284,26495,0,749579,31,amarela,<NA>,31,None,None,None,Blastocisto Grau 4 - B - C,<NA>,RAQUI METÁLICA (5),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,3.2,D6,7.28,B2E,1,27,None,<NA>,5e9f2d67899e009c9e1d548aef59171d,2025-07-21 21:38:57
3,76555,214264,26495,0,749579,11,amarela,<NA>,11,None,None,None,Blastocisto Grau 6 - B - C,<NA>,RAQUI METÁLICA (5),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,3.3,D6,8.36,B2E,1,27,None,<NA>,57718ca1eb9d403e68c984449f0bd42a,2025-07-21 21:38:57
4,76554,214263,26495,0,749579,10,amarela,<NA>,10,None,None,None,Blastocisto Grau 5 - B - B,<NA>,RAQUI METÁLICA (5),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,4.7,D6,2.05,B2E,1,27,None,<NA>,6b61d2cf3ed38fa03c085ba09ad84877,2025-07-21 21:38:57
5,76536,214291,26495,0,749579,38,amarela,<NA>,38,None,None,None,Blastocisto Grau 4 - B - B,<NA>,RAQUI METÁLICA (4),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,6.9,D5,1.16,B2E,5,44,None,<NA>,a20dac37e363719dd135788828c0478a,2025-07-21 21:38:57
6,76535,214290,26495,0,749579,37,amarela,<NA>,37,None,None,None,Blastocisto Grau 4 - A - B,<NA>,RAQUI METÁLICA (4),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,6.9,D5,8.03,B2E,5,44,None,<NA>,e86c0959fd1774dcc1f1cfdf9b5cb14a,2025-07-21 21:38:57
7,76534,214288,26495,0,749579,35,amarela,<NA>,35,None,None,None,Blastocisto Grau 5 - B - A,<NA>,RAQUI METÁLICA (4),None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,<NA>,None,None,None,None,None,None,None,None,None,Sim,Não analisado,8,D5,7.89,B2E,5,44,None,<NA>,7704041be026d7e0ee0c728c2fb5dcda,2025-07-21 21:38:57
8,76533,214286,26495,0,749579,33,amarela,<NA>,33,None,None,None,Blastocisto Grau 5 - B - A,<NA>,RAQU

In [11]:
congelamentos_ovulos_df

,id,CodCongelamento,Unidade,prontuario,paciente,Data,Hora,Ciclo,condicoes_amostra,empresa_transporte,clinica_origem,responsavel_recebimento,responsavel_recebimento_data,responsavel_armazenamento,responsavel_armazenamento_data,NOvulos,NPailletes,Identificacao,Tambor,Cane,Cane2,Tecnica,Motivo,Observacoes,BiologoResponsavel,BiologoFIV,BiologoFIV2,status_financeiro,hash,extraction_timestamp
0,11376,O3254/23,5,749579,esposa,2023-09-05,1900-01-01 11:30:00,2119/23,None,None,None,<NA>,NaT,<NA>,NaT,26,8,1242,B1O,4,48 e 49,Vitrificação,Social,2 RAQUES METÁLICAS,<NA>,Temizio Rodrigues Pereira,None,None,31338e5e58902835b7b0d7fb2b8988e9,2025-07-21 21:38:49


In [12]:
descongelamentos_ovulos_df 


,id,CodDescongelamento,Unidade,prontuario,doadora,DataCongelamento,DataDescongelamento,Ciclo,Identificacao,CodCongelamento,Tambor,Cane,PailletesDescongeladas,Tecnica,Observacoes,BiologoFIV,BiologoFIV2,hash,extraction_timestamp
0,3430,DO461/23,5,749579,0,2023-09-05,2023-11-03,2119/23,1242,O3254/23,B1O,4,3,DESVITRIFICAÇÃO,None,None,None,a3fdde75618a83cb975c82250515fe3f,2025-07-21 21:38:53


In [13]:
diario_df

,Cliente,Nome,Paciente,Nom Paciente,prontuario,DT Emissao,Descricao,Qntd.,Total,Descrição Gerencial,Loja,Tipo da nota,Numero,Serie Docto.,NF Eletr.,Vend. 1,Médico,Cliente_totvs,Operador,Produto,Valor Mercadoria,Custo,Custo Unit,Desconto,Unidade,Mês,Ano,Cta-Ctbl,Cta-Ctbl Eugin,Interno/Externo,Descrição Mapping Actividad,Ciclos,Qnt Cons.,Grp,Descr.TES,Lead Time,Data do Ciclo,Fez Ciclo?,line_number,extraction_timestamp,file_name
0,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-09-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,27178,3,26431,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,24444,20251013_151805,Mescladas2 (2).xlsx
1,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-08-18,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,26043,3,25292,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,479992,20251013_151805,Mescladas2 (2).xlsx
2,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-07-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,24766,3,24023,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,478529,20251013_151805,Mescladas2 (2).xlsx
3,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-06-16,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,23613,3,22885,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,477259,20251013_151805,Mescladas2 (2).xlsx
4,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-05-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,22393,3,21670,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,475878,20251013_151805,Mescladas2 (2).xlsx
5,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-04-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,21273,3,20550,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,474650,20251013_151805,Mescladas2 (2).xlsx
6,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-03-17,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,20212,3,19492,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,473476,20251013_151805,Mescladas2 (2).xlsx
7,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-02-17,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,19207,3,18488,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,472352,20251013_151805,Mescladas2 (2).xlsx
8,796333,THIAGO COSTA A,817292,DANIELA ALVES AMORIM PELUCIO,749579,2025-01-15,MENSALIDADE CRIOPRESERVACAO DE EMBRIAO -,1.0,125.00,Criopreservação,30101,508,18015,3,17306,0,CRIOPRESERVACAO,862243,,S3566.39DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,31110100020,,,Anuidade - Criopreservação,0,0,7,PRESTACAO DE SERVICO,None,NaT,False,470978,20251013_151805,Mescladas2 (2).xlsx
9,749582,GILBERTO RIBEI,817292,DANIELA ALVES AMORIM PELUCIO,749579,2024-12-19,UTROGESTAN 200MG C 14 CAPSULAS,6.0,474.00,Estimulação,30101,510,17310,3,16603,109,LIVIA OLIVEIRA MUNHOZ,879432,,0000000004632DF,0.0,0.0,0.0,0.00,FIV Brasilia,0,0,11310100005,,,Estimulação,0,

In [14]:
diario_df.groupby('Descricao').agg(
    {'Cta-Ctbl': 'count', 'Total': 'sum'}).sort_values(by='Total', ascending=False)

,Cta-Ctbl,Total
Descricao,,
ANALISE DE EMBRIOES PGT-A,4,18819.23
FOT - FERTILIZACAO DE OVULOS CONGELADOS,1,12120.70
FIV - COLETA MEDICOS INTERNOS - 3° CICLO,1,11144.79
FIV (COLETA) - MEDICOS INTERNOS,1,10157.00
FET - EMBRIAO EXCEDENTE - 2º CICLO,1,9787.71
FIV (INDUCAO) - MEDICOS INTERNOS,1,8706.00
FIV - INDUCAO MEDICOS INTERNOS - 3° CICL,1,8644.92
EMBRYOSCOPE,3,8068.00
BIOPSIA POR EMBRIOES - MEDICOS INTERNOS,3,8040.37


In [15]:
# Connect to huntington_data_lake for embryo timeline
data_lake_path = '../../database/huntington_data_lake.duckdb'

# Query embryo freeze timeline for a specific prontuario
prontuario = 182925  # Change this to any prontuario you want to analyze

with db.connect(data_lake_path, read_only=True) as conn_data_lake:
    embryo_timeline_df = conn_data_lake.execute(f"""
        SELECT *
        FROM gold.embryo_freeze_timeline
        WHERE prontuario = {prontuario}
        ORDER BY period_month DESC
    """).fetchdf()

print(f"=== EMBRYO FREEZE TIMELINE FOR PRONTUARIO {prontuario} ===")
print(f"Total records: {len(embryo_timeline_df)}")
print(embryo_timeline_df.to_string(index=False))


=== EMBRYO FREEZE TIMELINE FOR PRONTUARIO 182925 ===
Total records: 7
 prontuario period_month  embryos_frozen_count  embryos_unfrozen_count  embryos_discarded_count  embryos_storage_balance
     182925      2025-01                   0.0                       1                        0                      0.0
     182925      2024-08                   0.0                       1                        0                      1.0
     182925      2022-08                   0.0                       1                        0                      2.0
     182925      2022-01                   0.0                       1                        0                      3.0
     182925      2021-12                   4.0                       0                        0                      4.0
     182925      2021-10                   1.0                       1                        0                      0.0
     182925      2021-07                   5.0                       0             